# Arbitrage Order Generation Notebook

This notebook makes orders. The find_arbitrage method is where logic is being stored.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
options : pd.DataFrame = pd.read_csv("data/cleaned_options_data.csv")
options["day"] = options["ts_recv"].apply(lambda x: x.split("T")[0])

In [ ]:
start_date : datetime = datetime(2024, 1, 1)
end_date : datetime = datetime(2024, 3, 30)

In [ ]:
current_date = start_date
arbitrage_cutoff = 0.75 # cents
orders = []

options : pd.DataFrame = pd.read_csv("data/cleaned_options_data.csv")
options["day"] = options["ts_recv"].apply(lambda x: x.split("T")[0])
options['day'] = options.loc[:, 'day'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

def find_arbitrage(instrument_data: pd.DataFrame):
    global orders
    instrument_data = instrument_data.sort_values(['bid_px_00'])
    best_bid = instrument_data[['bid_px_00', 'bid_sz_00', 'symbol', 'ts_recv']].reset_index(drop=True)
    instrument_data = instrument_data.sort_values(['ask_px_00'], ascending=False)
    best_ask = instrument_data[['ask_px_00', 'ask_sz_00', 'symbol', 'ts_recv']].reset_index(drop=True)
    
    # concaenate the two dfs horizontally
    concatenated_df = pd.concat([best_bid, best_ask], axis=1)
    bid_pointer = 0
    ask_pointer = len(concatenated_df) - 1
    
    # display(concatenated_df)
    
    while bid_pointer < len(concatenated_df) and ask_pointer >= 0:
        if concatenated_df.at[bid_pointer, 'bid_px_00'] < concatenated_df.at[ask_pointer, 'ask_px_00'] + arbitrage_cutoff:
            bid_pointer += 1
        else:
            bid_size = concatenated_df.at[bid_pointer, 'bid_sz_00']
            ask_size = concatenated_df.at[ask_pointer, 'ask_sz_00']
            order_size = min(bid_size, ask_size)
            curr_bid, curr_ask = bid_pointer, ask_pointer
            bid_time, ask_time = best_bid.at[curr_bid, 'ts_recv'], best_ask.at[curr_ask, 'ts_recv']
            
            if ask_time > bid_time:
                bid_pointer += 1
                continue
            
            
            if bid_size > ask_size:
                concatenated_df.at[bid_pointer, 'bid_sz_00'] -= order_size
                ask_pointer -= 1
            elif bid_size < ask_size:
                concatenated_df.at[ask_pointer, 'ask_sz_00'] -= order_size
                bid_pointer += 1
            else:
                concatenated_df.at[bid_pointer, 'bid_sz_00'] -= order_size
                concatenated_df.at[ask_pointer, 'ask_sz_00'] -= order_size
                bid_pointer += 1
                ask_pointer -= 1
                        
            if order_size > 0:
                order_size = min(order_size, 50)
                orders.append({
                    "datetime" : best_bid.at[curr_bid, 'ts_recv'],
                    "option_symbol" : best_bid.at[curr_bid, 'symbol'],
                    "action" : "S",
                    "order_size" : order_size
                })
                orders.append({
                    "datetime" : best_ask.at[curr_ask, 'ts_recv'],
                    "option_symbol" : best_ask.at[curr_ask, 'symbol'],
                    "action" : "B",
                    "order_size" : order_size
                })
                break
                
                
    return None

while current_date <= end_date:
    current_data = options.loc[options['day'] == current_date, :]
    print(current_date, current_data.shape[0])
    if current_data.shape[0] == 0:
        current_date = current_date + pd.DateOffset(days=1)
        continue
    
    
    current_data = current_data.sort_values(['instrument_id', 'bid_px_00'])
    instrument_ids = current_data['instrument_id'].unique()
    
    for instrument_id in instrument_ids:
        instrument_data = current_data.loc[current_data['instrument_id'] == instrument_id, :]

        find_arbitrage(instrument_data) # global appends ot orders
    
    current_date = current_date + pd.DateOffset(days=1)